In [4]:
import pandas as pd
import numpy as np
import json
import re

# Reading data

In [5]:
bar_fixtures = pd.read_csv('data/raw/Barcelona - Fixture.csv')[['Date', 'Comp', 'Result', 'GF', 'GA', 'Opponent', 'Poss', 'Formation']]
bar_passing = pd.read_csv('data/raw/Barcelona - Passing.csv')[['Date',  'Cmp', 'Att', 'Cmp%', 'TotDist', 'PrgDist', 'Cmp.1', 'Att.1', 'Cmp%.1', 'Cmp.2', 'Att.2', 'Cmp%.2', 'Cmp.3', 'Att.3', 'Cmp%.3' ]]
ch_fixtures = pd.read_csv('data/raw/Chelsea - Fixture.csv')[['Date', 'Comp', 'Result', 'GF', 'GA', 'Opponent', 'Poss', 'Formation']]
ch_passing = pd.read_csv('data/raw/Chelsea - Passing.csv')[['Date', 'Cmp', 'Att', 'Cmp%', 'TotDist', 'PrgDist', 'Cmp.1', 'Att.1', 'Cmp%.1', 'Cmp.2', 'Att.2', 'Cmp%.2', 'Cmp.3', 'Att.3', 'Cmp%.3' ]]

In [6]:
bar_fixtures.shape, bar_passing.shape

((60, 8), (60, 15))

In [7]:
ch_fixtures.shape, ch_passing.shape

((63, 8), (63, 15))

In [8]:
bar_fixtures.head(1)

,Date,Comp,Result,GF,GA,Opponent,Poss,Formation
0,2018-08-12,Supercopa de España,W,2,1,Sevilla,69,4-3-3


In [9]:
bar_passing.head(1)

,Date,Cmp,Att,Cmp%,TotDist,PrgDist,Cmp.1,Att.1,Cmp%.1,Cmp.2,Att.2,Cmp%.2,Cmp.3,Att.3,Cmp%.3
0,2018-08-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
bar_fixtures.isna().sum(), bar_passing.isna().sum()

(Date         0
 Comp         0
 Result       0
 GF           0
 GA           0
 Opponent     0
 Poss         0
 Formation    0
 dtype: int64,
 Date        0
 Cmp        10
 Att        10
 Cmp%       10
 TotDist    10
 PrgDist    10
 Cmp.1      10
 Att.1      10
 Cmp%.1     10
 Cmp.2      10
 Att.2      10
 Cmp%.2     10
 Cmp.3      10
 Att.3      10
 Cmp%.3     10
 dtype: int64)

In [11]:
ch_fixtures.isna().sum(), ch_passing.isna().sum()

(Date         0
 Comp         0
 Result       0
 GF           0
 GA           0
 Opponent     0
 Poss         0
 Formation    0
 dtype: int64,
 Date        0
 Cmp        10
 Att        10
 Cmp%       10
 TotDist    10
 PrgDist    10
 Cmp.1      10
 Att.1      10
 Cmp%.1     10
 Cmp.2      10
 Att.2      10
 Cmp%.2     10
 Cmp.3      10
 Att.3      10
 Cmp%.3     10
 dtype: int64)

In [12]:
bar_merged = bar_fixtures.merge(bar_passing, on = 'Date')
bar_merged.isna().sum()

Date          0
Comp          0
Result        0
GF            0
GA            0
Opponent      0
Poss          0
Formation     0
Cmp          10
Att          10
Cmp%         10
TotDist      10
PrgDist      10
Cmp.1        10
Att.1        10
Cmp%.1       10
Cmp.2        10
Att.2        10
Cmp%.2       10
Cmp.3        10
Att.3        10
Cmp%.3       10
dtype: int64

In [13]:
ch_merged = ch_fixtures.merge(ch_passing, on = 'Date')
ch_merged.isna().sum()

Date          0
Comp          0
Result        0
GF            0
GA            0
Opponent      0
Poss          0
Formation     0
Cmp          10
Att          10
Cmp%         10
TotDist      10
PrgDist      10
Cmp.1        10
Att.1        10
Cmp%.1       10
Cmp.2        10
Att.2        10
Cmp%.2       10
Cmp.3        10
Att.3        10
Cmp%.3       10
dtype: int64

In [14]:
bar_merged.shape, ch_merged.shape

((60, 22), (63, 22))

# Chart 1

In [15]:
bar_merged['game_idx'] = bar_merged.index
bar_merged.Result = bar_merged.Result.apply(lambda x: 1 if x == 'W' else 0)
first_chart_cols = ['game_idx', 'Poss', 'GF', 'Result']

bar_merged_win = bar_merged[first_chart_cols].query('Result == "W"').reset_index(drop = True)
bar_merged_los = bar_merged[first_chart_cols].query('Result != "W"').reset_index(drop = True)

bar_merged[first_chart_cols].head(2)

,game_idx,Poss,GF,Result
0,0,69,2,1
1,1,79,3,1


In [16]:
ch_merged['game_idx'] = ch_merged.index
first_chart_cols = ['game_idx', 'Poss', 'GF', 'Result']
ch_merged.Result = ch_merged.Result.apply(lambda x: 1 if x == 'W' else 0)

ch_merged_win = ch_merged[first_chart_cols].query('Result == "W"').reset_index(drop = True)
ch_merged_los = ch_merged[first_chart_cols].query('Result != "W"').reset_index(drop = True)

ch_merged[first_chart_cols].head(2)

,game_idx,Poss,GF,Result
0,0,47,0,0
1,1,63,3,1


In [18]:
possession_json = {}
possession_json['bar'] = [{
    'x': int(bar_merged.game_idx.values[i]),
    'y': int(bar_merged.Poss.values[i]),
    'z': 0,
    'Win': int(bar_merged.Result.values[i])
} for i in range(bar_merged.shape[0])]

possession_json['ch'] = [{
    'x': int(ch_merged.game_idx.values[i]),
    'y': int(ch_merged.Poss.values[i]),
    'z': 0,
    'Win': int(ch_merged.Result.values[i])
} for i in range(ch_merged.shape[0])]

with open('data/possession.json', 'w') as outfile:
    json.dump(possession_json, outfile)

In [19]:
# ch_json = {}
# ch_json['games'] = [{
#     'x': int(ch_merged.game_idx.values[i]),
#     'y': int(ch_merged.Poss.values[i]),
#     'z': int(ch_merged.GF.values[i]),
#     'Win': int(ch_merged.Result.values[i])
# } for i in range(ch_merged.shape[0])]

# with open('data/processed/ch_json.txt', 'w') as outfile:
#     json.dump(ch_json, outfile)

In [16]:
# bar_json

# Chart 2

In [33]:
bar_value = pd.read_csv('data/raw/Barcelona - Value.csv')
ch_value = pd.read_csv('data/raw/Chelsea - Value.csv')

In [34]:
def parse_value(v):
    value = v.replace('$', '')
    multiplier = -1
    if 'Th.' in v:
        value = value.replace('Th.', '')
        multiplier = 1000
    elif 'm' in v:
        value = value.replace('m', '')
        multiplier = 1000000
    return float(value) * multiplier

def parse_name(name):
    return name.split('\n')[0].strip()

def remove_tails(df):
    top_11_df = df.iloc[:11].copy()
    other_total = sum(df.iloc[11:].Value.values)
    top_11_df = top_11_df.append({'Name': 'All Other', 'Birthday': '', 'Value': other_total}, ignore_index=True)
    return top_11_df

In [35]:
bar_value.Value = bar_value.Value.apply(parse_value)
ch_value.Value = ch_value.Value.apply(parse_value)

bar_value.Name = bar_value.Name.apply(parse_name)
ch_value.Name = ch_value.Name.apply(parse_name)

bar_value = bar_value.sort_values(by = 'Value', ascending = False).reset_index(drop = True)
ch_value = ch_value.sort_values(by = 'Value', ascending = False).reset_index(drop = True)

ch_value = remove_tails(ch_value)
bar_value = remove_tails(bar_value)

In [36]:
ch_value.head()

,Name,Birthday,Value
0,Eden Hazard,"Jan 7, 1991 (27)",165000000.0
1,N'Golo Kanté,"Mar 29, 1991 (27)",110000000.0
2,Jorginho,"Dec 20, 1991 (26)",71500000.0
3,Kepa,"Oct 3, 1994 (23)",66000000.0
4,Antonio Rüdiger,"Mar 3, 1993 (25)",55000000.0


In [37]:
value_json = {}

value_json['ch'] = [{
    'name': (ch_value.Name.values[i]),
    'y': int(ch_value.Value.values[i])
} for i in range(ch_value.shape[0])]

value_json['bar'] = [{
    'name': (bar_value.Name.values[i]),
    'y': int(bar_value.Value.values[i])
} for i in range(bar_value.shape[0])]

with open('data/value.json', 'w') as outfile:
    json.dump(value_json, outfile)

In [201]:
# bar_value_json = {}
# bar_value_json['players'] = [{
#     'name': (bar_value.Name.values[i]),
#     'y': int(bar_value.Value.values[i])
# } for i in range(bar_value.shape[0])]
# with open('data/processed/bar_value.json', 'w') as outfile:
#     json.dump(bar_value_json, outfile)

# Radar Chart

In [50]:
from functools import reduce

In [51]:
def handle_date(d):
    return str(pd.to_datetime(d))
def merge_two_table(t1, t2):
    return t1.merge(t2, on = 'Date', how = 'outer')

ch_gk = pd.read_csv('data/raw/Chelsea - GK.csv')[['Date', 'Saves', 'Save%']]
ch_def = pd.read_csv('data/raw/Chelsea - Defense.csv')[['Date', 'Tkl', 'TklW', 'Press', '%']]
ch_shoot = pd.read_csv('data/raw/Chelsea - Shoot.csv')[['Date', 'Sh', 'SoT', 'SoT%', 'G/SoT']]
ch_pass = pd.read_csv('data/raw/Chelsea - Passing.csv')[['Date', 'Att', 'Cmp%']]

ch_gk.columns = ['Date', 'Saves', 'Saves_perc']
ch_gk['Saves_perc'] = ch_gk['Saves_perc'].fillna(ch_gk.Saves_perc.mean())
ch_def.columns = ['Date', 'Tk', 'Tk_perc', 'Press', 'Press_perc']
ch_def['Tk_perc'] = ch_def.apply(lambda s: s.Tk_perc / s.Tk, axis = 1)
ch_shoot.columns = ['Date', 'Sh', 'SoT', 'SoT_perc', 'Sh_perc']
ch_pass.columns = ['Date', 'Pass', 'Pass_perc']

ch_gk.Date = ch_gk.Date.apply(handle_date)
ch_def.Date = ch_def.Date.apply(handle_date)
ch_shoot.Date = ch_shoot.Date.apply(handle_date)
ch_pass.Date = ch_pass.Date.apply(handle_date)

In [52]:
print(ch_gk.isna().sum())
ch_gk.head(2)

Date          0
Saves         0
Saves_perc    0
dtype: int64


,Date,Saves,Saves_perc
0,2018-08-05 00:00:00,6,0.75
1,2018-08-11 00:00:00,2,1.00


In [53]:
print(ch_def.isna().sum())
ch_def.head(2)

Date           0
Tk            10
Tk_perc       10
Press         10
Press_perc    10
dtype: int64


,Date,Tk,Tk_perc,Press,Press_perc
0,2018-08-05 00:00:00,NaN,NaN,NaN,NaN
1,2018-08-11 00:00:00,13.0,0.538462,144.0,24.3


In [54]:
print(ch_shoot.isna().sum())
ch_shoot.head(2)

Date        0
Sh          0
SoT         0
SoT_perc    0
Sh_perc     1
dtype: int64


,Date,Sh,SoT,SoT_perc,Sh_perc
0,2018-08-05 00:00:00,6,2,33.3,0.00
1,2018-08-11 00:00:00,12,3,25.0,0.67


In [55]:
print(ch_pass.isna().sum())
ch_pass.head(2)

Date          0
Pass         10
Pass_perc    10
dtype: int64


,Date,Pass,Pass_perc
0,2018-08-05 00:00:00,NaN,NaN
1,2018-08-11 00:00:00,682.0,87.5


In [56]:
ch_merged_for_radar = reduce(merge_two_table, [ch_gk, ch_def, ch_shoot, ch_pass])
ch_merged_for_radar.head(2)

,Date,Saves,Saves_perc,Tk,Tk_perc,Press,Press_perc,Sh,SoT,SoT_perc,Sh_perc,Pass,Pass_perc
0,2018-08-05 00:00:00,6,0.75,NaN,NaN,NaN,NaN,6,2,33.3,0.00,NaN,NaN
1,2018-08-11 00:00:00,2,1.00,13.0,0.538462,144.0,24.3,12,3,25.0,0.67,682.0,87.5


In [57]:
ch_merged_for_radar.shape

(63, 13)

In [58]:
print('Number of rows without NaN:', ch_merged_for_radar.apply(lambda s: s.isna().sum() == 0, axis = 1).sum())

Number of rows without NaN: 53


In [59]:
ch_merged_for_radar = ch_merged_for_radar.dropna()
# ch_merged_for_radar.to_csv('data/ch_merged_for_radar.csv')

In [60]:
# ch_json_radar = {}
# ch_json_radar['games'] = [{
#     'x': int(ch_merged_for_radar.game_idx.values[i]),
#     'y': int(ch_merged.Poss.values[i]),
#     'z': int(ch_merged.GF.values[i]),
#     'Win': int(ch_merged.Result.values[i])
# } for i in range(ch_merged.shape[0])]

# with open('data/processed/ch_json_radar.txt', 'w') as outfile:
#     json.dump(ch_json_radar, outfile)

In [61]:
# Barcelona

In [62]:
bar_gk = pd.read_csv('data/raw/Barcelona - GK.csv')[['Date', 'Saves', 'Save%']]
bar_def = pd.read_csv('data/raw/Barcelona - Defense.csv')[['Date', 'Tkl', 'TklW', 'Press', '%']]
bar_shoot = pd.read_csv('data/raw/Barcelona - Shoot.csv')[['Date', 'Sh', 'SoT', 'SoT%', 'G/SoT']]
bar_pass = pd.read_csv('data/raw/Barcelona - Passing.csv')[['Date', 'Att', 'Cmp%']]

bar_gk.columns = ['Date', 'Saves', 'Saves_perc']
bar_gk['Saves_perc'] = bar_gk['Saves_perc'].fillna(bar_gk.Saves_perc.mean())
bar_def.columns = ['Date', 'Tk', 'Tk_perc', 'Press', 'Press_perc']
bar_def['Tk_perc'] = bar_def.apply(lambda s: s.Tk_perc / s.Tk, axis = 1)
bar_shoot.columns = ['Date', 'Sh', 'SoT', 'SoT_perc', 'Sh_perc']
bar_pass.columns = ['Date', 'Pass', 'Pass_perc']

bar_gk.Date = bar_gk.Date.apply(handle_date)
bar_def.Date = bar_def.Date.apply(handle_date)
bar_shoot.Date = bar_shoot.Date.apply(handle_date)
bar_pass.Date = bar_pass.Date.apply(handle_date)

In [73]:
bar_merged_for_radar = reduce(merge_two_table, [bar_gk, bar_def, bar_shoot, bar_pass])
bar_merged_for_radar.head(2)

,Date,Saves,Saves_perc,Tk,Tk_perc,Press,Press_perc,Sh,SoT,SoT_perc,Sh_perc,Pass,Pass_perc
0,2018-08-12 00:00:00,3,0.6670,NaN,NaN,NaN,NaN,17,8,47.1,0.25,NaN,NaN
1,2018-08-18 00:00:00,0,0.7372,6.0,0.5,108.0,52.8,25,9,36.0,0.33,890.0,89.0


In [74]:
print('Number of rows without NaN:', bar_merged_for_radar.apply(lambda s: s.isna().sum() == 0, axis = 1).sum())

Number of rows without NaN: 50


In [75]:
bar_merged_for_radar = bar_merged_for_radar.dropna()
bar_merged_for_radar.to_csv('data/bar_merged_for_radar.csv')

In [78]:
def final_processing(df):
    
    mean_df = df.mean()
    mean_df.Saves_perc = mean_df.Saves_perc * 100
    mean_df.Tk_perc = mean_df.Tk_perc * 100
    mean_df.Sh_perc = mean_df.Sh_perc * 100
    mean_df = mean_df.round(2)
    summ_stats_perc = mean_df[['SoT_perc', 'Pass_perc', 'Saves_perc', 'Tk_perc', 'Press_perc', 'Sh_perc']]
    summ_stats_perc.index = ['Shoot On Target', 'Pass Success', 'Saves', 'Tackle Success', 'Press Success', 'Shoot to Goal']
    
    summ_stats = mean_df[['SoT', 'Pass', 'Saves', 'Tk', 'Press', 'Sh']]
    return {
        'summ_stats': [list(summ_stats.values), list(summ_stats.keys())], 
        'summ_stats_perc': [list(summ_stats_perc.values), list(summ_stats_perc.keys())],
    }
ch_radar_dict = final_processing(ch_merged_for_radar)
bar_radar_dict = final_processing(bar_merged_for_radar)

Shoot On Target    34.42
Pass Success       86.08
Saves              67.99
Tackle Success     66.00
Press Success      29.77
Shoot to Goal      31.70
dtype: float64

Shoot On Target    43.96
Pass Success       87.53
Saves              74.87
Tackle Success     66.17
Press Success      28.95
Shoot to Goal      31.94
dtype: float64

In [79]:
radar_json = {}
radar_json['ch'] = ch_radar_dict
radar_json['bar'] = bar_radar_dict

with open('data/radar.json', 'w') as outfile:
    json.dump(radar_json, outfile)

In [68]:
ch_radar_dict

{'summ_stats': [[5.26, 697.72, 1.89, 15.68, 162.51, 15.91],
  ['SoT', 'Pass', 'Saves', 'Tk', 'Press', 'Sh']],
 'summ_stats_perc': [[34.42, 86.08, 67.99, 66.0, 29.77, 31.7],
  ['SoT_perc', 'Pass_perc', 'Saves_perc', 'Tk_perc', 'Press_perc', 'Sh_perc']]}

In [69]:
radar_json

{'ch': {'summ_stats': [[5.26, 697.72, 1.89, 15.68, 162.51, 15.91],
   ['SoT', 'Pass', 'Saves', 'Tk', 'Press', 'Sh']],
  'summ_stats_perc': [[34.42, 86.08, 67.99, 66.0, 29.77, 31.7],
   ['SoT_perc',
    'Pass_perc',
    'Saves_perc',
    'Tk_perc',
    'Press_perc',
    'Sh_perc']]},
 'bar': {'summ_stats': [[6.46, 705.1, 2.6, 15.3, 166.16, 15.06],
   ['SoT', 'Pass', 'Saves', 'Tk', 'Press', 'Sh']],
  'summ_stats_perc': [[43.96, 87.53, 74.87, 66.17, 28.95, 31.94],
   ['SoT_perc',
    'Pass_perc',
    'Saves_perc',
    'Tk_perc',
    'Press_perc',
    'Sh_perc']]}}

In [182]:
# def calc_standarlized_score(raw_score, sequence):
    

In [181]:
keys = ['SoT', 'Pass', 'Saves', 'Tk', 'Press', 'Sh']
for i in range(6):
    ch_stats = radar_json['ch']['summ_stats'][0][i]
    bar_stats = radar_json['bar']['summ_stats'][0][i]
    all_stats = np.array(list(bar_merged_for_radar[keys[i]].values) + list(ch_merged_for_radar[keys[i]].values))
    bar_score = sum(all_stats < bar_stats)
    ch_score = sum(all_stats < ch_stats)
    print(bar_score, ch_score)

67 50
45 45
62 40
60 60
59 55
52 52


In [180]:
len(all_stats)

103

# Parliment chart

In [38]:
parli_json = {}

In [46]:
bar_parliment_data = [
    ['Messi', 36],
    ['Suarez', 21],
    ['Dembele', 8],
    ['Coutinho', 5],
    ['Pique', 4],
    ['Others', 14]
]

ch_parliment_data = [
    ['Hazard', 16],
    ['Pedro', 8],
    ['Loftus-Cheek', 6],
    ['Higuain', 5],
    ['Morata', 5],
    ['Others', 21]
]

In [47]:
parli_json['bar'] = bar_parliment_data
parli_json['ch'] = ch_parliment_data

In [48]:
with open('data/parliment.json', 'w') as outfile:
    json.dump(parli_json, outfile)

# Heatmap

In [80]:
heatmap_json = {}

In [87]:
bar_heatmap_data = [
    ['Lionel Messi', 13],
    ['Jordi Alba', 8],
    ['Arturo Vidal', 7],
    ['Sergi Roberto', 7],
    ['Luis Suárez', 6],
    ['Ivan Rakitic', 5],
    ['Ousmane Dembélé', 5],
    ['Others', 10]
]
tmp_lst = []
for pair in bar_heatmap_data:
    tmp_lst.append({'name': pair[0], 'value': pair[1], 'colorValue': pair[1]})
bar_heatmap_data = tmp_lst

ch_heatmap_data = [
    ['Eden Hazard', 15],
    ['Willian', 6],
    ['César Azpilicueta', 5],
    ['Ross Barkley', 5],
    ['Marcos Alonso', 4],
    ["N'Golo Kanté", 4],
    ['Olivier Giroud', 4],
    ['Others', 9]
]
tmp_lst = []
for pair in ch_heatmap_data:
    tmp_lst.append({'name': pair[0], 'value': pair[1], 'colorValue': pair[1]})
ch_heatmap_data = tmp_lst

In [88]:
parli_json['bar'] = bar_heatmap_data
parli_json['ch'] = ch_heatmap_data

In [89]:
with open('data/heatmap.json', 'w') as outfile:
    json.dump(parli_json, outfile)